In [3]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### download (tutorial data)

In [4]:
import requests
request = requests.get("https://drive.google.com/uc?export=download&id=1wHt8PsMLsfX5yNSqrt2fSTcb8LEiclcf")
with open("data.zip", "wb") as file:
    file.write(request.content)

# Unzip data
import zipfile
with zipfile.ZipFile('data.zip') as zip:
    zip.extractall('data')

### load (tutorial data)

In [5]:
 # Load data and set labels
data_complaint = pd.read_csv('data/complaint1700.csv')
data_complaint['label'] = 0
data_non_complaint = pd.read_csv('data/noncomplaint1700.csv')
data_non_complaint['label'] = 1

# Concatenate complaining and non-complaining data
data = pd.concat([data_complaint, data_non_complaint], axis=0).reset_index(drop=True)

# Drop 'airline' column
data.drop(['airline'], inplace=True, axis=1)

# Display 5 random samples
data.sample(5)

,id,tweet,label
1777,8522,@Jlee0097 @ScottsdaleToDo @willbasset @dolphin...,1
704,149129,Soooo I spent $179 to upgrade and be comfy on ...,0
1507,123235,Feeling slightly frustrated @AmericanAir Fligh...,0
349,155838,Hey @united. You better get some training for ...,0
1548,53116,Hey @JetBlue I'm at Laguardia waiting to board...,0


### Train test split (tutorial data)

In [6]:
from sklearn.model_selection import train_test_split

X = data.tweet.values
y = data.label.values

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.1, random_state=2020)

### Test data (tutorial data)

In [10]:
# Load test data
test_data = pd.read_csv('data/test_data.csv')

# Keep important columns
test_data = test_data[['id', 'tweet']]

# Display 5 samples from the test data
test_data.sample(5)

,id,tweet
196,7528,#fail @SouthwestAir restricting triple points ...
1281,50183,@americanair flight AA 5385 should have left a...
3971,151419,@BridgetPhetasy I had to fly @united in March ...
3806,144567,I usually like @SouthwestAir but today's fligh...
950,37789,Worst. MT@BostonDotCom: JetBlue doesn't love u...


### Check if GPU is available

In [70]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


### Data pre-processing (tutorial data)

In [12]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

### print sample processed text (tutorial data)

In [13]:
# Print sentence 0
print('Original: ', X[0])
print('Processed: ', text_preprocessing(X[0]))

Original:  @united I'm having issues. Yesterday I rebooked for 24 hours after I was supposed to fly, now I can't log on &amp; check in. Can you help?
Processed:  I'm having issues. Yesterday I rebooked for 24 hours after I was supposed to fly, now I can't log on & check in. Can you help?


### !!! LOAD OUR DATA !!!

In [140]:
# load CSV into DataFrame
df_tweets = pd.read_csv("Data/Twitter_Data.csv")

# rename columns
df_tweets.columns = ["text", "label"]

# drop NaN
df_tweets.dropna(inplace=True)

# create X and y
X_tweets, y_tweets = df_tweets.text.values, df_tweets.label.values

# small dataset to !!! TEST STUFF WITH !!!
X_tweets_small, y_tweets_small = X_tweets[:2500], y_tweets[:2500]

In [108]:
# encode all tweets
all_tweets_encoded = [tokenizer.encode(text, add_special_tokens = True) for text in X_tweets]

# get max length
max_len_tweets = max([len(text) for text in all_tweets_encoded])

# print max length
print(f"Tweet max lengt: {max_len_tweets}")

Tweet max lengt: 102


In [109]:
MAX_LEN = 128

### Tokenizer

In [116]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text = text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens = True,        # Add `[CLS]` and `[SEP]`
            max_length = MAX_LEN,             # Max length to truncate/pad
            pad_to_max_length = True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask = True,     # Return attention mask
            truncation = True               # added by Christopher, was getting warnings
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

### Preprocessing (tokenize) tutorial data

In [115]:
# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X_tweets[0]])[0].squeeze().numpy())
print('Original: ', X_tweets[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Original:  when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples
Token IDs:  [101, 2043, 16913, 2072, 5763, 1523, 6263, 2231, 4555, 10615, 1524, 3517, 2032, 4088, 1996, 3697, 3105, 29455, 1996, 2110, 2339, 2515, 2202, 2086, 2131, 3425, 2110, 2323, 1998, 2025, 2449, 1998, 2323, 6164, 8827, 2271, 1998, 8436, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...
